Primero, se importan los datos del archivo `"A1.4 Vino Tinto.csv"` al ambiente de trabajo y se revisan las dimensiones del `DataFrame` , despues se imprimen dimensiones y las primeras 5 filas de los datos.


In [48]:
import pandas as pd

df = pd.read_csv("A1.4 Vino Tinto.csv")
print("Cantidad de datos y variables:",df.shape)
df.head()

Cantidad de datos y variables: (1599, 12)


,acidezFija,acidezVolatil,acidoCitrico,azucarResidual,cloruros,dioxidoAzufreLibre,dioxidoAzufreTotal,densidad,pH,sulfatos,alcohol,calidad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Utilizando la función `sample` de `pandas`, dividimos la base de datos en dos subconjuntos, `train` (80% de los datos) y `test` (20% restante). 

In [46]:
import pandas as pd

train = df.sample(frac=0.8, random_state=42)
test = df.drop(train.index)                 

print("Train:",train.shape)
print("Test:",test.shape)

print(train.shape[0] + test.shape[0])

(1279, 12)
(320, 12)
1599


Ahora implementaremos la seleccion hacia adelante, la cual nos permite seleccionar la cantidad de variables mas optima para usar en nuestro modelo. Esto se basa en ir usando las variables una por una y ademas, usar las combinaciones con otras variables, asi hasta tener un conjunto donde usamos todas las variables. En este caso se usaran diferentes parametros dentor de la funcion `SequentialFeatureSelector`. Aqui se ecuentran el 
- **`estimator`**: Define que modelo se usara, en este caso `LinearRegression`.
- **`k_features`**: El rango del número de variables a seleccionar. Por ejemplo `(2, 8)`.
- **`forward`**: Determina si es seleccion hacia adelante (True) o hacia atras (false).
- **`scoring`**: La métrica para evaluar el modelo, en este caso sera R cuadrada.
- **`cv`**: Número de particiones para validación cruzada.

In [59]:
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector

forwardSel = SequentialFeatureSelector(estimator = LinearRegression(), k_features = (2,8), forward = True, scoring = "r2", cv = 10)
forwardSel = forwardSel.fit(X,Y)

print("Indices de las características seleccionadas:", forwardSel.k_feature_idx_)
print("Características seleccionadas:", list(forwardSel.k_feature_names_))

Indices de las características seleccionadas: (1, 4, 5, 6, 8, 9, 10)
Características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


Despues de hacer la seleccion hacia adelante, entrenamos un modelo con dichas variables selecciondas. Estas las usaremos la funcion `predict` para predecir nuestro muestreo `Train` y usaremos la R cuadrada para hacer el anaisis.

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

fowardVar = list(forwardSel.k_feature_names_)

xTrain = train[fowardVar]
yTrain = train.calidad

model = LinearRegression()
model.fit(xTrain, yTrain)

xTest = test[fowardVar]
yTest = test.calidad

yPred = model.predict(xTest)

r2 = r2_score(yTest, yPred)
print("R2 :", r2)

R2 : 0.4242250763140887


Despues, haremos una seleccion hacia atras sobre la misma seleccion hacia adelante que ya hicimos, esto tambien se le conoce como seleccion mixta. A diferencia de la anterior, en este caso los rangos de `k_features` seran de 2 a 5 y como queremos hacerlo hacia atras, el parametro forward sera Falso. Una vez hecho esto, repetiremos la creacion del modelo pero ahora con las variables seleccionadas por la seleccion mixta.

In [61]:
backwardSel = SequentialFeatureSelector(estimator = LinearRegression(), k_features = (2,5), forward = False, scoring = "r2", cv = 10)
backwardSel = backwardSel.fit(X,Y)

print("Indices de las características seleccionadas:", backwardSel.k_feature_idx_)
print("Características seleccionadas:", list(backwardSel.k_feature_names_))

backwardVar = list(backwardSel.k_feature_names_)

xTrain = train[backwardVar]
yTrain = train.calidad

model = LinearRegression()
model.fit(xTrain, yTrain)

xTest = test[backwardVar]
yTest = test.calidad

yPred = model.predict(xTest)

r2 = r2_score(yTest, yPred)
print("R2 :", r2)

Indices de las características seleccionadas: (1, 4, 6, 9, 10)
Características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']
R2 : 0.4394113030555302


Despues de realizar ambos modelos, se puede observer que ambas R's cuadradas tienen valores muy similares, apesar de dividir los ambientes de forma randomizada, los valores tienden a tener diferencias de decimales. Con esto en mente, podemos llegar a la conclusion de que el modelo de seleccion mixta tendria mejores resultados, a pesar de ser una diferencia minima, tambien podemos considerar que se estan usando menos variables y consigue de mejor manera predecir lo que tambien hace el modelo de seleccion hacia adelante pero con mas variables. Por lo que, considero que el segundo modelo, o sea el de seleccion mixta, es mejor.